In [79]:
#encoding=utf-8
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB


import pickle
import sys,os
from all_model_py import CutDebt, IDClassifier, IfKnowDebtor, Installment, WillingToPay, ConfirmLoan


# load data

In [89]:
def sub_df(df,sets,target='label'):
    result = pd.DataFrame()
    for each in sets:
        result = pd.concat([result,df[df[target]==each]])
#     print(result[target].value_counts())
    return result

model_list = {'CutDebt':CutDebt,
              'IDClassifier':IDClassifier,
              'IfKnowDebtor':IfKnowDebtor,
              'Installment':Installment,
              'ConfirmLoan':ConfirmLoan,
              'WillingToPay':WillingToPay}

others = pd.read_csv('../../data/others/cleaned_mock_up_data.csv')
other_matrix = pd.read_csv('../../data/others/strategy_mat.csv')
target = 'label'
save_path = '../../savedModel/{}/'
for model in model_list:
    
    df = pd.read_csv('../../data/{}/cleaned_mock_up_data.csv'.format(model))
    other_label = max(set(df.label))
    # filter out other label
    df = df[df.label != other_label]
    # get availabel other labels
    other_set = set(other_matrix[other_matrix[model]==0].label.values)
    ava_others = sub_df(others,other_set)
    ava_others[target] = other_label
    ava_others = ava_others.rename({'text':'split_text'},axis=1)
    df = pd.concat([df,ava_others],sort=True)
    df = df.sample(frac=1).reset_index(drop=True)
    print('=====  {} ======='.format(model))
    print(df.label.value_counts())
    print('begin training!')
    
    # get tfidf
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(df.split_text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(df.split_text)
    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, df.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, df.label)
    
    
    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, df.label)
    print('finish training')
    
    other = pickle.load(open('../../savedModel/others/{}/{}_other.pickle'.format(model,model),'rb'))
    result = model_list[model](svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer, other=other,  jieba_path='../WordCut/userdict.txt')
    pickle.dump(result, open(save_path.format(model) + model + '.pickle', "wb"))
    
    


=====  CutDebt =======
0    4105
2    2819
1    2144
Name: label, dtype: int64
begin training!
fitting phrase
transform phrase
finish training
=====  IDClassifier =======
2    2585
0    1446
1    1296
Name: label, dtype: int64
begin training!
fitting phrase
transform phrase
finish training
=====  IfKnowDebtor =======
2    2585
0    1483
1    1177
Name: label, dtype: int64
begin training!
fitting phrase
transform phrase
finish training
=====  Installment =======
0    4105
2    2819
1    2144
Name: label, dtype: int64
begin training!
fitting phrase
transform phrase
finish training
=====  ConfirmLoan =======
2    2819
0     580
1     553
Name: label, dtype: int64
begin training!
fitting phrase
transform phrase
finish training
=====  WillingToPay =======
3    2191
0    2106
1    2095
2    2069
Name: label, dtype: int64
begin training!
fitting phrase
transform phrase
finish training


In [90]:
idc = pickle.load(open("../../savedModel/IDClassifier/IDClassifier.pickle", 'rb'))
cutd = pickle.load(open("../../savedModel/CutDebt/CutDebt.pickle", 'rb'))
ifk = pickle.load(open("../../savedModel/IfKnowDebtor/IfKnowDebtor.pickle", 'rb'))
will = pickle.load(open("../../savedModel/WillingToPay/WillingToPay.pickle", 'rb'))
inst = pickle.load(open("../../savedModel/Installment/Installment.pickle", 'rb'))
conf = pickle.load(open("../../savedModel/ConfirmLoan/ConfirmLoan.pickle", 'rb'))

In [94]:
import jieba
jieba_path='../WordCut/userdict.txt'
jieba.load_userdict(jieba_path)


In [99]:
sentence = '我艹你妈'
sentence = jieba.cut(sentence, cut_all = False)
sentence = ' '.join(sentence)
sentence

'我 艹 你 妈'

In [91]:
idc.classify('我日')

{'label': 0, 'pred_prob': array([[0.8137873 , 0.01224582, 0.17396687],
        [0.77023028, 0.03276042, 0.1970093 ],
        [0.36572189, 0.20767766, 0.42660045]]), 'av_pred': array([0.64991316, 0.08422797, 0.26585887])}

In [92]:
ifk.classify('我日')

{'label': 104, 'pred_prob': array([[0.2801312 , 0.03821481, 0.68165399],
        [0.47673074, 0.02345738, 0.49981189],
        [0.43348973, 0.07660823, 0.48990204]]), 'av_pred': array([0.39678389, 0.04609347, 0.55712264])}

In [25]:
conf.classify('我只借过5000')

{'label': 1, 'pred_prob': array([[0.03012535, 0.83516209, 0.13471256],
        [0.19332805, 0.47517069, 0.33150125],
        [0.18685578, 0.5741989 , 0.23894532]]), 'av_pred': array([0.13676973, 0.62817723, 0.23505304]), 'time_extract': []}